### Importing necessary libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as pt
import seaborn as sns
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn import datasets

### Loading the data

In [2]:
#Importing titanic data from the required location
boston = datasets.load_boston()
X, y = boston.data, boston.target
print(X.shape, y.shape)

(506, 13) (506,)


In [3]:
### Data Description
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [4]:
input_ads = pd.DataFrame(X, columns = boston.feature_names)
input_ads = pd.concat([input_ads, pd.DataFrame(y, columns = ['MEDV'])], axis = 1)
input_ads.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Description of the target variable

In [5]:
input_ads['MEDV'].describe()

count    506.000000
mean      22.532806
std        9.197104
min        5.000000
25%       17.025000
50%       21.200000
75%       25.000000
max       50.000000
Name: MEDV, dtype: float64

### Null Check

In [6]:
pd.DataFrame(input_ads.isnull().sum()).T

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Train and Test set preparation and conversion to arrays

In [7]:
X = input_ads[[cols for cols in list(input_ads.columns) if 'MEDV' not in cols]]
y = input_ads['MEDV']
X, X_test, y, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

#--------------------------------------------------------------------------------------
# Scaling the datasets

scaler = StandardScaler()

X_arr = scaler.fit_transform(X)
X_test_arr = scaler.transform(X_test)


y_arr = np.array(y).reshape(X_arr.shape[0], 1)
y_test_arr = np.array(y_test).reshape(X_test_arr.shape[0], 1)

#-------------------------------------------------------------------------------------

print(X_arr.shape)
print(X_test_arr.shape)
print(y_arr.shape)
print(y_test_arr.shape)

(354, 13)
(152, 13)
(354, 1)
(152, 1)


## Manual Gradient Boosting Code

### UDF definition that are required for training GradientBoosting

In [8]:
from sklearn.tree import DecisionTreeRegressor

#---------------------------------------------------------------------------------------------------------
#Loss func
def loss_calc(y_true,y_pred):
    
    loss = (1/len(y_true)) * 0.5*np.sum(np.square(y_true-y_pred))
        
    return loss

#---------------------------------------------------------------------------------------------------------
#Gradient Calc
def gradient_calc(y_true,y_pred):
    
    grad = -(y_true-y_pred)
    
    return grad

#---------------------------------------------------------------------------------------------------------
#The base estimator
def tree_creator(r_state,X,y):
    
    d_tree = DecisionTreeRegressor(random_state=r_state,criterion='mse',
                                    max_depth=2,min_samples_split=5,
                                    min_samples_leaf=5,max_features=3)
    d_tree.fit(X,y)
    
    return d_tree

#---------------------------------------------------------------------------------------------------------
#Predicting through gradient boosting regression
def predict_grad_boost(models_tray,alpha,test_x=X_test_arr,train_y=y_arr):
    
    initial_pred = np.array([np.mean(train_y)] * len(test_x))
        
    final_pred = initial_pred.reshape(len(initial_pred),1)
    #print(final_pred.shape)
    
    for i in range(len(models_tray)):
        
        model = models_tray[i]
        temp_pred = (model.predict(test_x)).reshape(len(test_x),1)
        #print(temp_pred.shape)
        final_pred -= alpha * temp_pred
    
    return final_pred


### UDF for Gradient Boosting training

In [9]:
def grad_boost_train(train_x,train_y,alpha=0.01,r_state=100,n_iters=101):

    model_tray = [] #Tray to collect the trained boosted stage estimators
    loss_counter = [] #Tray for loss capture

    
    initial_pred = np.array([np.mean(train_y)] * len(train_y))

    print('Initial val :',initial_pred.shape)
    model_pred = initial_pred.reshape(len(initial_pred),1)

    for epoch in range(n_iters): #Unit iteration

        if epoch%100==0:
            print('#---------- Epoch number :',epoch,' -----------#')
        
        #Calculating loss
        loss = loss_calc(y_true=train_y,
                         y_pred=model_pred)

        loss_counter.append(loss)
        
        #Calculating the gradient (residuals)
        grads = gradient_calc(y_true=train_y,
                              y_pred=model_pred)
        #print(grads.shape)
        #Building the regression tree on the gradient (residuals)
        tree_grad = tree_creator(r_state=r_state,
                                 X=train_x,
                                 y=grads)
        #print(train_x.shape)
        #print(tree_grad.predict(train_x).shape)
        
        #Predicting the residuals according to the tree fit above
        pred_m = (tree_grad.predict(train_x)).reshape(len(train_x),1)
        
        #Updating model through learning rate
        model_pred -= alpha * pred_m
        
        #Appending the model into tray
        model_tray.append(tree_grad)
        
    return model_tray,loss_counter,initial_pred

### Invoking the gradient boosting training UDF

In [10]:
#-----------------------------------------------------------------------------------------------------------------------
#Defining some hyper-params
n_estimators = 1001 #No of boosting steps
alpha =0.01 #Learning rate

#Training gradient boosting regression
models_list,loss_counter,initial_pred = grad_boost_train(train_x=X_arr,
                                                         train_y=y_arr,
                                                         alpha=alpha,
                                                         r_state=100,
                                                         n_iters=n_estimators)

Initial val : (354,)
#---------- Epoch number : 0  -----------#
#---------- Epoch number : 100  -----------#
#---------- Epoch number : 200  -----------#
#---------- Epoch number : 300  -----------#
#---------- Epoch number : 400  -----------#
#---------- Epoch number : 500  -----------#
#---------- Epoch number : 600  -----------#
#---------- Epoch number : 700  -----------#
#---------- Epoch number : 800  -----------#
#---------- Epoch number : 900  -----------#
#---------- Epoch number : 1000  -----------#


### Plotting Loss Curve, there should be a decrease in training loss over boosting rounds

In [11]:
# sns.set_style('darkgrid')
# ax = sns.lineplot(x=range(n_estimators),y=loss_counter)
# ax.set(xlabel='Number of boosting rounds',ylabel='Loss',title='Loss vs Boosting rounds plot')

### Predicting on the test dataset using the manual training above (Only the trained residual models are passed)

In [12]:
manual_gbm_pred = predict_grad_boost(models_tray=models_list, #Passing the fitted estimators into the predict function
                                     alpha=alpha, #The alpha val used during training
                                     test_x=X_test_arr) #Test dataset
manual_gbm_pred.shape

(152, 1)

### Evaluating the predictions from the manual gradient boosting model

In [13]:
print('MSE of test set :',mean_squared_error(y_test_arr,manual_gbm_pred))
print('RMSE of test set :',np.sqrt(mean_squared_error(y_test_arr,manual_gbm_pred)))

MSE of test set : 47.786260272468155
RMSE of test set : 6.912760683870674


### Benchmarking against sklearn implementation of gradient boosting

In [14]:
from sklearn.ensemble import GradientBoostingRegressor

skl_gbm = GradientBoostingRegressor(random_state=100,n_estimators=1001,criterion='mse',
                                    max_depth=2,min_samples_split=5,
                                    min_samples_leaf=5,max_features=3)

#-------------------------------------------------------------------------------
skl_gbm.fit(X_arr,y_arr)
skl_pred = skl_gbm.predict(X_test_arr)

#-------------------------------------------------------------------------------
print('MSE of test set :',mean_squared_error(y_test_arr,skl_pred))
print('RMSE of test set :',np.sqrt(mean_squared_error(y_test_arr,skl_pred)))

MSE of test set : 17.40914943928571
RMSE of test set : 4.1724272838823335


### Comments
#### 1. The manual implementation of gradient boosting is providing comparable RMSE values compared to sklearn GBM

### END